In [ ]:
!pip install selenium webdriver-manager

In [1]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.common.exceptions import UnexpectedAlertPresentException
import re
import time


options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    driver.get("https://gcloud.csi.go.kr/cmq/main.do")
    print("\n[대기] 로그인 후 상세페이지의 '빨간 테두리' 영역이 보이면 Enter를 누르세요.")
    input(">> 준비 완료 후 Enter: ")

    print("\n" + "="*70)
    print(" [빨간 테두리 영역 정밀 스캔 시작]")
    print("="*70)

    # 1. 채취자/봉인자/참관자 영역 (data-name 속성 활용)
    print("\n[1. 인적 사항 정보]")
    people_sections = ["pick_user_text", "seal_user_text", "attnd_user_text"]
    for section in people_sections:
        try:
            # data-name 속성을 가진 행을 찾아 그 줄의 전체 텍스트 출력
            row = driver.find_element(By.XPATH, f"//th[@data-name='{section}']/..")
            print(f"> {row.text.strip().replace('\n', ' | ')}")
        except:
            continue

    # 2. 공사개요 영역 (고유 ID값 직접 지칭)
    print("\n[2. 공사개요 상세]")
    construction_fields = {
        "공사명": "bizNm",
        "현장소재지": "addr",
        "착공일": "bgngDt",
        "준공예정일": "cmplPlanDt"
    }
    
    for label, element_id in construction_fields.items():
        try:
            value = driver.find_element(By.ID, element_id).text.strip()
            print(f"> {label} : {value}")
        except:
            print(f"> {label} : 데이터를 찾을 수 없음")

    # 3. 기타 항목 (성과 이용 목적, 발주자, 시공자 등)
    print("\n[3. 기타 항목]")
    # 텍스트 매칭을 통해 제목 바로 옆의 값을 가져옵니다.
    other_labels = ["성과 이용 목적", "발주자", "시공자", "국가중요시설 여부"]
    for label in other_labels:
        try:
            val = driver.find_element(By.XPATH, f"//th[contains(text(), '{label}')]/following-sibling::td").text.strip()
            print(f"> {label} : {val}")
        except:
            continue

    print("\n" + "="*70)
    print(" 모든 정밀 스캔이 완료되었습니다.")

except Exception as e:
    print(f"\n[오류] 추출 중 문제가 발생했습니다: {e}")



[대기] 로그인 후 상세페이지의 '빨간 테두리' 영역이 보이면 Enter를 누르세요.

 [빨간 테두리 영역 정밀 스캔 시작]

[1. 인적 사항 정보]
> 채취자 소속 | 현대엔지니어링 주식회사 | 담당업무 | 품질관리 | 성명 | 정광훈 (서명 완료)

[2. 공사개요 상세]
> 공사명 : 부천대장 공종주택지구 조성공사 2공구
> 현장소재지 : 경기도 부천시 오정구 벌말로 43
> 착공일 : 2024-04-25
> 준공예정일 : 데이터를 찾을 수 없음

[3. 기타 항목]
> 성과 이용 목적 : 품질시험·검사 대행
> 발주자 : 한국토지주택공사 인천지역본부 (담당자: 최기찬)
> 시공자 : 현대엔지니어링 주식회사
> 국가중요시설 여부 : 해당없음

 모든 정밀 스캔이 완료되었습니다.
